# naver movie sentimental analysis

In [1]:
!pip install konlpy > /dev/null

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

## 1. Data Preprocessing

- train data set

In [5]:
# Check for duplicates
train_df.document.nunique()

146182

In [6]:
# Exclude duplicate data
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [7]:
# check null data
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [8]:
# remove null data
train_df.dropna(how='any', inplace=True)
train_df.shape

(146182, 3)

In [9]:
# distribution of pos, neg label
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- train data set

In [10]:
test_df.document.nunique()

49157

In [11]:
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [12]:
test_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [13]:
test_df.dropna(how='any', inplace=True)
test_df.shape

(49157, 3)

In [14]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

## 2. text preprocessing

- train dataset

In [15]:
# remove except hangul, blank
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [16]:
train_df['document'].replace('', np.nan, inplace=True)
train_df.document.isnull().sum()

391

In [17]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145791, 3)

- test dataset

In [18]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
test_df['document'].replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48995, 3)

## 3. Process Hangul

In [20]:
from konlpy.tag import Okt
okt = Okt()

In [23]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 한국어불용어100.txt to 한국어불용어100.txt


In [25]:
df = pd.read_csv(filename, sep='\s+', header=None)
df.head()

,0,1,2
0,이,VCP,0.018280
1,있,VA,0.011699
2,하,VV,0.009774
3,것,NNB,0.009733
4,들,XSN,0.006898


In [26]:
# 품사 무시하고 중복 배제
stopwords = set(df[0])
len(stopwords)

94

In [32]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [33]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [34]:
' '.join(okt.morphs(text, stem=True))

'교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정'

In [38]:
from tqdm.notebook import tqdm
str_train = []
for sentence in tqdm(train_df.document):
  morphs = okt.morphs(sentence.strip(), stem=True)
  temp_str = ' '.join(word for word in morphs if word not in stopwords)
  str_train.append(temp_str)

  0%|          | 0/145791 [00:00<?, ?it/s]

In [39]:
str_train[:5]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다',
 '교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정',
 '사이 몬페 의 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 하다 커스틴 던스트 너무나도 이쁘다 보이다']

In [40]:
str_test = []
for sentence in tqdm(test_df.document):
  morphs = okt.morphs(sentence.strip(), stem=True)
  temp_str = ' '.join(word for word in morphs if word not in stopwords)
  str_test.append(temp_str)

  0%|          | 0/48995 [00:00<?, ?it/s]

In [41]:
y_train = train_df.label.values # np array
y_test = test_df.label.values

## 4. Feature transform
- CountVectorizer

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(ngram_range=(1,2))

In [43]:
cvect.fit(str_train)
X_train = cvect.transform(str_train)
X_test = cvect.transform(str_test)
X_train.shape, X_test.shape

((145791, 575305), (48995, 575305))

## 5. model train/predict/valuate

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [45]:
lr = LogisticRegression(max_iter=500)
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
pred = lr.predict(X_test)

In [47]:
accuracy_score(y_test, pred)

0.8451882845188284

## 6. real test

In [73]:
review1 = '뭐 액션은 말이 필요없는 수준이고 마지막 클라이맥스 씬에서 스케일이나 영상미가 예술이었음ㅋㅋㅋ 페이즈 4에서 아주 중요한 영화인 듯하니 아이맥스로 꼭 보시길!'
review2 = '1점을 주는 이유는 언론이나 영화 유튜버들이 너무 이 영화를 띄워주었기 때문이다.걍 액션영화 그 이상 그 이하도 아닌 영화'

In [74]:
# dont convert
review2.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣] ', '')

'1점을 주는 이유는 언론이나 영화 유튜버들이 너무 이 영화를 띄워주었기 때문이다.걍 액션영화 그 이상 그 이하도 아닌 영화'

In [75]:
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ] ', '', review1)
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ] ', '', review2)

In [76]:
reviews = [review1, review2]
review_list = []
for review in tqdm(reviews):
  review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review)
  morphs = okt.morphs(review.strip(), stem=True)
  temp_str = ' '.join(word for word in morphs if word not in stopwords)
  review_list.append(temp_str)

  0%|          | 0/2 [00:00<?, ?it/s]

In [77]:
review_list

['뭐 액션 은 필요없다 수준 이고 마지막 클라이맥스 씬 에서 스케일 이나 영상 미가 예술 이다 ㅋㅋㅋ 페이 즈 에서 아주 중요하다 영화인 듯 하다 아이맥스 로 꼭 보다',
 '을 주다 이유 는 언론 이나 영화 유튜버들 너무 영화 를 띄우다 이다 걍 액션영화 이상 이하 도 아니다 영화']

In [78]:
review_cv = cvect.transform(review_list)
pred = lr.predict(review_cv)
pred

array([1, 0])